# PDF에서 문장이 이상하게 잘린 경우 문장 교종해주는 코드
190903
최종

# 잘린 첫글자가 숫자인 경우는 안됨....
1. 여러 문장이 한줄에 입력된 경우 잘라주기
2. 소문자, 대문자, 숫자로 시작하는 문장의 index
3. PDF로 문장이 짤린 경우 이어주기
4. txt로 저장('~_fixed.txt')

In [1]:
import spacy, os, glob, re
nlp = spacy.load('en_core_web_sm')

In [2]:
def split_long_one_line(total_line):
    ## 1. 여러 문장이 한줄에 입력된 경우 잘라주기
    fixed_long_line = []
    for long_line in total_line:
        # spacy 문장 나누기로 쪼개서 문장별 correction 해서 저장
        text_sentences = nlp(long_line)
        # 쪼개진 문장 갯수
        for idx, sentence in enumerate(text_sentences.sents):
            len_sentences = idx + 1
        # 쪼개질게 없으면 바로 통과
        if len_sentences == 1:
            fixed_long_line.append(long_line)
        elif len_sentences > 1:
            for sentence in text_sentences.sents:
                fixed_long_line.append(sentence.text) # 잘린건 뒤에 '\n가 없어서 붙여줘야한다'
    return fixed_long_line

def merge_sentences(fixed_long_line):
    ## 2. 소문자, 대문자, 숫자로 시작하는 문장의 index
    lower_idx, upper_idx, not_eng_idx = [], [], []
    for line_idx, line in enumerate(fixed_long_line):
        if line[0].islower():
            lower_idx.append(line_idx)
        elif line[0].isupper():
            upper_idx.append(line_idx)
        else:
            not_eng_idx.append(line_idx)

    ## 3. PDF로 문장이 짤린 경우 이어주기
    fixed_short_line = []
    count = 0
    while count < len(fixed_long_line)-1:
        # 시작이 대문자고 다음이 소문자면
        if fixed_long_line[count][0].isupper() and fixed_long_line[count+1][0].islower():
            # 소문자가 몇개 연달아 나오는지 센다
            count_lower = 0
            while 1:
                if count+count_lower+1 in lower_idx:
                    count_lower += 1
                else:
                    break
            # 대문자로 시작하는 문장
            plus_line = fixed_long_line[count].strip()+' '  # 좌우 자르고 이어주기
            # 대문자 뒤 소문자 연속하는 부분 idx for문
            for ii in range(count+1, count+ count_lower+1):
                plus_line += fixed_long_line[ii].strip()+' ' # 좌우 자르고 이어주기

            # spacy 문장 나누기로 쪼개서 문장별 correction 해서 저장
            text_sentences = nlp(plus_line)
            for sentence in text_sentences.sents:
                fixed_short_line.append(sentence.text.strip())
            count += count_lower+1
        else:
            # '\n' 후에 '\n' 나오면 다음줄 건너띄자
            if fixed_long_line[count] == '\n' and fixed_long_line[count+1] == '\n':
                fixed_short_line.append(fixed_long_line[count].strip())
                count += 2
            else:
                fixed_short_line.append(fixed_long_line[count].strip())
                count += 1
    return fixed_short_line

def save_fixed_as_txt(path, fixed_short_line):
    ## 4. txt로 저장
    f = open(path.split('.')[0]+'_fixed.txt', 'w', encoding='UTF-8')
    for fixed_line in fixed_short_line:
        # 4글자 미만은 버리자. 0은 '\n'을 살리기 위해
        if len(fixed_line) == 0 or len(fixed_line) > 3:
            f.write(fixed_line+'\n')  # 왼쪽만 공백 제거,  strip으로 하면 '\n'도 날아간다
    f.close()
    
def sentence_corrector(path):
    # 데이타 읽어오기
    test_data = open(path, 'r', encoding='UTF-8').readlines()
    total_line = []
    for line in test_data:
        ## 정규표현식으로 에러나는 부분 교체해주기
        # “”는 문장쪼개기에서 에러나므로 '"' 로 치환
        line = re.sub(r'[“”]', '"', line)
        line = re.sub(r'[\xa0]', '' , line)  # 이상한 빈칸 없애기 
        line = re.sub(r' . .'  , '' , line)  # . . . . .3 목차 지우기
        total_line.append(line)
    ## 1. 여러 문장이 한줄에 입력된 경우 잘라주기
    fixed_long_line = split_long_one_line(total_line)
    
    ## 2. 소문자, 대문자, 숫자로 시작하는 문장의 index
    ## 3. PDF로 문장이 짤린 경우 이어주기
    fixed_short_line = merge_sentences(fixed_long_line)
    ## 4. txt로 저장
    save_fixed_as_txt(path, fixed_short_line)

In [3]:
dirname = '20190902_txt'
report_company_list = os.listdir(dirname)

In [ ]:
for report_company in report_company_list:
    print('report_company :', report_company)
    report_list = glob.glob(os.path.join(dirname, report_company, "*.txt"))
    report_list = [name for name in report_list if '_fixed' not in name]  # _fixed 안된것만 하기
    
    for report_idx, report in enumerate(report_list):
        print('%d / %d'% (report_idx+1, len(report_list)))
        ## !! sentence corrector 실행!!
        sentence_corrector(report)

report_company : FE
1 / 7
2 / 7
3 / 7
4 / 7
5 / 7
6 / 7
7 / 7
report_company : RF
1 / 31
2 / 31
3 / 31
4 / 31
5 / 31
6 / 31
7 / 31
8 / 31
9 / 31
10 / 31
11 / 31
12 / 31
13 / 31
14 / 31
15 / 31
16 / 31
17 / 31
18 / 31
19 / 31
20 / 31
21 / 31
22 / 31
23 / 31
24 / 31
25 / 31
26 / 31
27 / 31
28 / 31
29 / 31
30 / 31
31 / 31
report_company : SM
1 / 8
2 / 8
3 / 8
4 / 8
5 / 8
6 / 8
7 / 8
8 / 8
report_company : TM
1 / 21
2 / 21
3 / 21
4 / 21
5 / 21
6 / 21
7 / 21
8 / 21
9 / 21
10 / 21
11 / 21
12 / 21
